# OTUs


We will now cluster sequences and assign taxonomy.

## Merge forward and backward reads
As the first step, we will merge forward and backward read sequences into one. The resultant sequences will be used for next steps.

```{.bash}
qiime vsearch merge-pairs \
--i-demultiplexed-seqs paired-end-demux-trimmed.qza \
--o-merged-sequences demux-joined.qza \
--o-unmerged-sequences demux-unjoined.qza

```

## Quality control of merged sequences
Now we will use merged pairs and apply quality filtering.

```{.bash}
qiime quality-filter q-score \
–-i-demux demux-joined.qza \
--p-min-quality 20 \
–-o-filtered-sequences demux-joined-filtered.qza
–-o-filter-stats demux-joined-filter-stats.qza
```


## Denoise using deblur


```{.bash}
qiime deblur denoise-16S \
  --i-demultiplexed-seqs demux-joined-filtered.qza \
  --p-trim-length 200 \
  --p-sample-stats \
  --o-representative-sequences rep-seqs.qza \
  --o-table de_table.qza \
  --o-stats deblur-stats.qza
```

# References

1. https://docs.qiime2.org/2024.10/tutorials/read-joining/